## Demand Analysis
<table><tr><td><img src="Images/dem.jpg"></td></tr></table>
 


#### Installing Plotly and Cufflinks for Interactive Plots

In [ ]:
#!pip install plotly v==3.0.1

In [ ]:
#!pip install chart-studio

In [ ]:
#!pip install cufflinks

### Importing the Neccessary Packages 

In [ ]:
#%matplotlib notebook
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from chart_studio.plotly import plot, iplot
import cufflinks as cf
from IPython.display import display
import plotly.offline
cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)


### Loading Data and Pre Processing 

In [ ]:
#Reading the Data into pandas DataFrame
df=pd.read_csv('Demand_Analysis.csv')

In [ ]:
df.head()

In [ ]:
#Renaming the Column "Hour of Week" to have no spaces
df.rename(columns={"Hour of Week": "Hour_of_Week"},inplace=True)

Since the Data has Hours of Week, i Need to Extract the Week Day based on 24 hours Each, So it will be easier to perform Deman Analysis based on Daily and Hourly observations So for Pre Processing i need to
- **Create WeekDay Column**
- **Extract Hours of Day From Hours of Week**
- **GroupBy both WeekDay and Hour of Day to get patterns**
- **Check Demand in both Percentage and Absolute Values**

In [ ]:
#Extracting WeekDay from hours of Week
df['Week_Day']=df['Hour_of_Week'].apply(lambda x : math.trunc(x/24))

There was no information as to which week day the first 24 hours represent. So i am just assuming the week starts with Monday. My whole Analysis could be Either termed wrong or right based on this Assumption. If this information was provided the Analysis would have been more accurate

In [ ]:
# Creating a Function to Name the WeekDays into their Respective Days. Starting From Monday
def Weekday(row):
 if   row['Week_Day']==0  :
      return 'Monday'
 elif row['Week_Day']==1 :
      return 'Tuesday'
 elif row['Week_Day']==2 :
      return 'Wednesday'
 elif row['Week_Day']==3 :
      return 'Thursday'
 elif row['Week_Day']==4 :
      return 'Friday'
 elif row['Week_Day']==5 :
      return 'Saturday'
 elif row['Week_Day']==6 :
      return 'Sunday'


In [ ]:
#Applying the Function on the Week Day column that i created
df['Week_Day'] = df.apply (lambda row: Weekday(row),axis=1)

**I'm going to convert the hours of Week Column to Hours of day so i can Group the data together and see the pattern based on each hour of each day. Im doing this by taking modulo of each value in the column with 24**

In [ ]:
#Convert Hours into 24 hours for each day
df['Hour_of_Week']=df['Hour_of_Week'].apply(lambda x: x%24)

In [ ]:
#Renaming the Column Hour of Week to Hour of Day
df.rename(columns={"Hour_of_Week": "Hours_of_Day"},inplace=True)

In [ ]:
df.head(1)

In [ ]:
df.tail(3)

Once the Data was Clean i Loaded it to Tableau to Visualise it further, i have attached some of the visuals to this Notebook and the Tableau File is provided in the folder

In [ ]:
# Save as CSV to load into Tableau
df.to_csv('Result.csv') 

## Demand Analysis

First of all i think visualizing the data by grouping it by Hours of Day and Week Day will show us a clear pattern of how the demand increases or decreases each week on hourly basis.
- **Creating the absolute value Demand Analysis Table**

First i will create a pivot table for my data using the Week Days and Hours of Day Attributes and Sum up the other columsn with regards to these

In [ ]:
#Using Pandas Crosstab Function to create a pivot tsble
dw=pd.crosstab([df.Hours_of_Day],[df.Week_Day], values=df.Reservations,aggfunc=np.sum)

In [ ]:
# Lets See how the head of the Pivot Table looks like
dw.head(3)

For more clarity i will create a heatmap of the Pivot Table to see how the values are Distributed across the weekdays and hours
- **Visualizing the PivotTable for Absolute Demand values using Seaborn Heatmap**

In [ ]:
# Plotting Heatmap for the Pivot Table
sns.set(font_scale=1.8)
fig, ax = plt.subplots(figsize=(25,15))
title= 'Number of Reservations Throughout The Week (Absolute Values)'
plt.title(title,fontsize=23)
sns.heatmap(pd.crosstab([df.Hours_of_Day],[df.Week_Day], values=df.Reservations,aggfunc=sum),
            cmap='BuPu', annot=True, cbar=False,fmt='g')


Although this is more clear, it is obviously hard to see any pattern from just these numbers, I will plot a spaghetti plot for each of the WeekDays and the Reservations throughout the week to see a proper pattern. 
- **Spaghetti Plot for Absolute Reservations Throughout the Week**

In [ ]:
#Grouping the Data by Hours of Day and Week Day and Summing the other values based on Hours and weekDays
df.groupby([df['Hours_of_Day'], 'Week_Day'])['Reservations'].sum().unstack().plot(title='Reservations Throughout the Week on Hourly Basis',
                                                        kind='line',colormap='RdYlGn',figsize=(20,8),grid=True, fontsize=10,
                                                        xticks=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23),linestyle='-', marker='o')


 The Above Plot is streched and plotted over 24 hours and for each day of the Week and shows a much clear pattern for demand throughout the week across 24 hours of each week day. Here are a couple of things we can observe from the plot
- **At Midnight**, There is still Demand at this hour of each day however, we see that during weekdays, there are more bookings as compared to the weekends.
- **After Midnight** , the Demand or number of Reservations start to Fall throughout the weekdays
- **6AM - 9AM**, The Demand seems to increase, showing that the reservations start early morning each day during this period
- **11AM - 12PM**, This period shows the Peak Demand where the most number of Reservations are made throughout the WeekDays. With the highest number of Reservations made at 12 o clock on a friday. This sudden peak on Friday at this time may indicate the People who are commuting for Friday Prayers. For these spikes Socar can introduce Either promos or can even raise the price
- **1PM - 11PM(Night)** There are stagnant high and lows during these hours, as the demand doesnt spike or falls below a certain level, indicating most of the reservations are during these hours.
- **Wednesday( Slow Day)** It can be seen that Wednesday is the slowest day for Socar Bookings, it is therefore essential to attract more customers on a wednesday using special day specific promos. ( I will plot a total booking bar plot per week day to see which day has more bookings and which one has least to see a better picture

To see more details i will next plot using Plotly for a more interactive plot that shows all values once you hover 

In [ ]:
dw.iplot()

- **Visualizing the PivotTable for Percentage Demand values using Seaborn Heatmap**

I will Further Analyse the Demand and reservations using the percentages of each day based on hours. Similar to Absolute values i will first create a pivot table and Visualize 

In [ ]:
dp=pd.crosstab(df.loc[:,'Hours_of_Day'],df.loc[:, 'Week_Day'], values=df.Reservations,aggfunc=np.sum,normalize='columns').applymap(lambda x: "{0:.0f}%".format(100*x)) 

In [ ]:
dp.head()

In [ ]:
sns.set(font_scale=1.8)
fig, ax = plt.subplots(figsize=(25,15))
title= 'Demand Analysis Percentages'
plt.title(title,fontsize=23)
sns.heatmap(pd.crosstab([df.Hours_of_Day],[df.Week_Day], values=df.Reservations,aggfunc=sum,normalize='columns'),
            cmap='PuBu', annot=True, cbar=False,fmt='.0%')

In [ ]:
dp.iplot(kind='line')

- **BarPlot For Total Reservations Grouped By WeekDay(Tableau)**

The Bar plot for total number of reservations based on week days in plotted below and we can see some important information from the figure below and the one above 
- **Wednesday is the Slowest**
I plot the total number of Reservations for each day using Tableau and we can clearly see that Wednesday has the least number of Reservations, indicating the the mid of the week is where customers dont use much of ours services, Introducing promos for this day can raise the number of reservations. 

- **Friday is Highest**
Furthermore, we can also see that most of the reservations and the highest are made during Fridays, this however can be due to the spike from 10AM Till 12PM which would indicate those people who are commuting to maybe Friday prayers using Socar services. Similarly there is a rise in Reservations during 5 o clock, 7 o clock during fridays, which may indicate those people who are leisurers and are travelling for the weekend to someplace. However, the reservations at 5 may also indicate those people who are commuting back to their homes.
- **Thurday Shows Spikes During Evening hours**
For Thursday being a weekday, it is quite surprising to see reservation spike during the hours of 2-4 and 6-7 and 10-11. For understanding this better more attributes are needed such as  which consumer group is booking at these hours. For now all i can think of is The first screening of new movies are on Thursdays, so people book for going to cinemas with friends and family. This however can explain the reservations during 6-7 and 10-11, but it still doesnt explain the spike during 2-4

<img src="Images/TotalReservation.png">


### Task 2:-  Document Approval Process Enhancement and conversion


To see the document approval patterns its first important for us to see when the majority of the signups to the app occur during the entire week. We plot this using Groupby Weekday on Tableau for a detailed Visual.

- **Visual for Average Signups per Weekday** We can see that majority of the signups occur during the Days Saturday, Sunday and Friday and they tend to decline throughout the week afterwards from monday onwards. This would also suggest that majority of the people upload their documents on Weekend and high sign up days. We see that the highest signups occur on Saturday and Followed by Sunday. Since Socar Document Approval can take upto 24 hours, It is therefore, essential to enhance this process inorder to increase number of bookings at this stage of the business

<img src="Images/Signups.png">

Now that we know which days most signups occur, Lets check the document approval pattern throughout the week 
- **Dotted/Marked Line plot for Total Number of Documents Approved Per Week Day( Plotted using Tableau**

<img src="Images/Approved.png">

We Can see that the Dcoument Approval Pattern is directly proportional to how many Signups were made each day. Lets us plot the Document Approved observations on Hourly basis of each week day
- **Document Approved on Hourly basis for each week day Spaghetti Plot**

In [ ]:
#Plotting Document Approved throughout Each Weekday based on hours of day
df.groupby([df['Hours_of_Day'], 'Week_Day'])['Doc_approved'].sum().unstack().plot(title='Documents Approved Throughout the Week on Hourly Basis',
                                                        kind='line',colormap='RdYlGn',figsize=(20,8),grid=True, fontsize=10,
                                                        xticks=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23),linestyle='-', marker='o')

We can clearly see that majority of the Documents are approved during the hours of 10 AM to 7PM But for a more detailed plot i will use Plotly to create an interactive plot

In [ ]:
#Plotting using Plotly for a more interactive Plot
ap=pd.crosstab([df.Hours_of_Day],[df.Week_Day], values=df.Doc_approved,aggfunc=np.sum)

In [ ]:
#Plotly Plot 
ap.iplot()

<img src="Images/plotlyapp.png">

The Above plot Clearly indicates that from 10 AM til 7PM the weekends are leading when it comes to maximum document approvals but that also maybe due to the fact that we have the highest signups during the weekend. It can be seen that there is a clear distinct path for the patterns of Weekdays and Weekend days. Where Weekend days such as Friday, Saturday and Sunday show a high spike during 10AM till 7PM. But to see a clear pattern of the spikes and the downfalls of Approved Documents we need to compare with Rejected Documents aswell 

- **Plotting Rejected Documents Per Hour for each WeekDay**

In [ ]:
#Plotting Document Rejected throughout Each Weekday based on hours of day
df.groupby([df['Hours_of_Day'], 'Week_Day'])['Doc_rejected'].sum().unstack().plot(title='Documents Rejected Throughout the Week on Hourly Basis',
                                                        kind='line',colormap='RdYlGn',figsize=(20,8),grid=True, fontsize=10,
                                                        xticks=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23),linestyle='-', marker='o')

In [ ]:
#Plotting using Plotly for a more interactive Plot
rj=pd.crosstab([df.Hours_of_Day],[df.Week_Day], values=df.Doc_rejected,aggfunc=np.sum)

In [ ]:
rj.iplot()

<img src="Images/Plotrej.png">

We see a sudden spike on sunday in documents rejected at 13 or 1 o clock in the afternoon this is exactly after the sudden spike in documents approved of the same day. This means that the fall off decent of the approved documents at 13 or 1 o clock was due to the large amount of documents being rejected. Documents can be rejected for any sort of reason. These can include
- **The Picture of the Document was not clear or Shaky**
- **The Document uploaded was not valid or Expired**
- **Approval Algorithm not flexible and faulty**


Regardless of the Reason, the document rejection has high chance of human error. Most of the times, the customers, are not as interested in uploading proper documentation, rather then getting started with the ride as fast as possible. A few Suggestions for improving the document approval process and conversion as follow:

### Suggestions for Document Approval Process Enhancement
- **Introduce a better guide or manual which include steps for properly uploading the document in a clear way**
- **Introduce point based authentication, where the customer will get points for uploading document properly, These point can later be redeemed during first ride**
- **Reason For Rejection should be provided to the customer in a clear way, and steps to get approval should also be mentioned either by email or by notification pop up** This atleast ensures the customer knows why the document was rejected and can follow some simple steps to rectify the error and reupload. In most cases if no proper reason is provided, customer tend to churn and loose interest
By making sure there is some kind of incentive for uploading the correct document carefully and in a clear way, the chance of human error can be reduced on the customers end, instead of focusing just on hurrying through the process and getting into a car, the customer will also focus on getting the points for proper authentication 

- **On developer end check and test the approval procedure to see if it randomly rejects correctly uploaded documents**, in some scenerios such as Grab Wallet, even though you upload proper documents in a proper manner, the system sometimes does not authenticate and no proper reasoning is given for rejection, this frustrates the customer and results in customer loosing interest in the product

- **If authentication process includes document evaluation being done by employees, then it is essential to make sure they are trained in a proper manner and taught to be unbiased in all regards** If an employee is biased towards certain group of people, or is in a mood that might effect his/her sentiment towards the cutomers, it may effect his judgment. This may result in loss of potential customers or wastage of time for gaining a potential sale or reservation in time 